In [74]:
import os
import sys
import json
import time 
import enum
import random
import inspect
import requests
import typing
import queue
import hashlib
import base64
import asyncio
import sqlite3

from dataclasses import dataclass, field
import typing
# from typing import Callable, List, Union, Set, Any

local_import_path = '/home/zu/x1ws/l6sk_python'

if local_import_path not in sys.path:
    sys.path.append(local_import_path)

In [75]:
from l6sk import knobman as km
# from l6sk.dbl import dao_sqlite
# from l6sk.dbl.dbl_dispatch import DBL_REQ
# from l6sk.dbl.dbl_api import DBL_API

In [76]:
# default log group.
dlg_conn = sqlite3.connect(":memory:", isolation_level=None)
dlg_cur = dlg_conn.cursor()
print(f"dbconn: {dlg_conn}")
print(f"cursor: {dlg_cur}")

dbconn: <sqlite3.Connection object at 0x7f8614a58d50>
cursor: <sqlite3.Cursor object at 0x7f85fecc8b90>


In [77]:
# executescript() to do multiple statements at once.
# but this does not allow accessing results
# sqlite is in process and there is no round trip penalty, no network, ...
# there is no point in doing multiple queries in one execute, you can just loop over your queries
# at least thats what they are saying. so dont expect an equivalent to mariadb multiple statements, ...

# test some competing table designs.

### Simplification hints/opinions:

- We are not trying to recreate ELK. We said ELK is too much. We want something lighter and more simple.
- Sort of like redis vs mongodb. They are both key value stores in a sense. 
but redis is redis is lighter, smaller, meant to be just a lookaside buffer, running on each instance, whereas
mongodb is meant to be the bigger scalabla db cluster that handles sharding, json features,
undo/redo log atomicity (at least for individual CRUD operation), reconciling data among nodes, leader election, ....


```text
- Logs go to "log_group". there is a default log_group if you didnt specify one.
- no hierarchical log groups and namespaces and what not. log groups are not like filesystem ie:
/my/cool/app/prod/....

just a flat namespace per l6sk instance.

- I think each log group should be a separate database/schema.
but this is the concern of the DB Layer.

- all names (table, schema, ..) must be < 63 chars in pg.
- start w/ a letter
- avoid SQL reserved keywords. 

- log group names can only be alphanumeric, start with letter, and can not contain 2 back to back underscore
- easier soln: prefix db/schema name w/ "l6sk_...." and impose 50 char limit, 
now the name must be in alphanumeric and thats it.



```




In [78]:
test_string = "zxcvbnmasdfghjklqwertyuiopZXCVBNMASDFGHJKLQWERTYUIOP1234567890-=_+|\{}':/><?'~`"
for ch in test_string:
    if not ch.isalnum():
        print(ch, end=" ")

print("\n\n\n")
for ch in test_string:
    if not ch.isalpha():
        print(ch, end=" ")

- = _ + | \ { } ' : / > < ? ' ~ ` 



1 2 3 4 5 6 7 8 9 0 - = _ + | \ { } ' : / > < ? ' ~ ` 

In [79]:
sess_id = base64.urlsafe_b64encode(os.urandom(18)).decode('ascii')
print(sess_id)

pxynCjrCohShI4X0lLhMzZs-


In [80]:
# ======================================================================================================================
# ======================================================================================================================
# =========================================================================================================== Log Record
@dataclass(frozen=False)
class LOG_RECORD:
    "A LOG_RECORD fully describes/tracks a single log message, and is generated for each log message."
    
    # client_ts: client timestamp. generated by client SDK at the time log msg was issued.
    client_ts: float = None
    
    # server timestamp, generated by the l6sk server upon reception. used for trim/rotate/discard/...
    # must not be null in the database.
    srv_ts: float = None
    
    # level attached to this log msg. i.e. if user said: log.dbg() then this is set to DBUG
    # for simplicity only allowed options are 'DBUG', 'INFO', 'WARN', 'ERRR', 'CRIT'
    lvl: str = None
        
    # optional. A subsystem name. None means no subsystem/main/generic. if you may want
    # to see/filter log msgs issued by separate subsystems
    # seperately you can use this. like get_logger(__file__)
    subsys: str = None
    
    # optional. everytime a client logger is init()ed, a new random session id will be created. 
    # l6sk user can also set a session id. its like a way to group things together within a log group.
    # useful if you ran the program 5 times, you can always see latest session (based on one of the ts)
    # and then go back and compare to the session 5 runs ago.
    session_id: str = None
    
    # caller information,, line number, function name, filename, modulename, package.
    lineno: str = None
    filename: str = None
    funcname: str = None
    
    # process info
    pname: str = None
    pid: str = None

    # thread info
    tname: str = None
    tid: str = None
    
    # the log msg issued to lg36.
    msg: str = None




In [107]:
refersh_schema_script = f"""
DROP TABLE IF EXISTS log_record;

-- log records
CREATE TABLE IF NOT EXISTS log_record(

    lrid INTEGER PRIMARY KEY AUTOINCREMENT NOT NULL,

    lvl TEXT NOT NULL,

    session_id TEXT,

    -- unix time
    srv_ts TEXT NOT NULL DEFAULT(strftime('%s','now')),

    -- client generated could be null
    client_ts TEXT,
    
    -- caller info, mostly optional, depends on client info gathering abilities
    lineno TEXT,
    filename TEXT,
    funcname TEXT,
    pid TEXT,
    pname TEXT,
    tid TEXT,
    tname TEXT,
    
    -- actual log msg. in case of fifo middleman, then this is a line of output.
    msg TEXT

    CHECK (lvl IN ('DBUG','INFO','WARN', 'ERRR', 'CRIT'))
);

"""


In [108]:
dlg_cur.executescript(refersh_schema_script)

In [109]:
dlg_cur.execute('INSERT INTO log_record(lrid, lvl, session_id, client_ts, msg) VALUES (100, "DBUG", "_sess_" , "1604852083", "Hi");')


In [110]:
# for client ts percision, its upto the client. for python client 4 makes sense

for _ in range(20):
    tmp_rec = [random.choice(['DBUG', 'INFO', "WARN", "ERRR", "CRIT"]), sess_id, f"{time.time():.4f}", "hello world"]
    dlg_cur.execute('INSERT INTO log_record(lvl, session_id, client_ts, msg) VALUES (?, ?, ?, ?);', tmp_rec)


In [111]:
dlg_cur.execute('SELECT * FROM log_record;')

for rec in dlg_cur.fetchall():
    print(rec)

(100, 'DBUG', '_sess_', '1604892734', '1604852083', None, None, None, None, None, None, None, 'Hi')
(101, 'WARN', 'pxynCjrCohShI4X0lLhMzZs-', '1604892735', '1604892735.1450', None, None, None, None, None, None, None, 'hello world')
(102, 'CRIT', 'pxynCjrCohShI4X0lLhMzZs-', '1604892735', '1604892735.1454', None, None, None, None, None, None, None, 'hello world')
(103, 'CRIT', 'pxynCjrCohShI4X0lLhMzZs-', '1604892735', '1604892735.1455', None, None, None, None, None, None, None, 'hello world')
(104, 'ERRR', 'pxynCjrCohShI4X0lLhMzZs-', '1604892735', '1604892735.1455', None, None, None, None, None, None, None, 'hello world')
(105, 'CRIT', 'pxynCjrCohShI4X0lLhMzZs-', '1604892735', '1604892735.1455', None, None, None, None, None, None, None, 'hello world')
(106, 'DBUG', 'pxynCjrCohShI4X0lLhMzZs-', '1604892735', '1604892735.1455', None, None, None, None, None, None, None, 'hello world')
(107, 'ERRR', 'pxynCjrCohShI4X0lLhMzZs-', '1604892735', '1604892735.1456', None, None, None, None, None, Non